### Understanding LangGraph 
LangGraph is a special LangChain-built libray that focuses on bulding intelligent AI Agent use graph. le, agentic
state machines

We need these prerequisite libraies to run a graph visualization libray (pygraphviz). we will use this libray during 
this notebook to understand the structure of our graphs but is not required to use langgraph.


In [3]:
# If you base on Mac os system
# !brew install python3 graphviz pkg-config

# If you base on ubuntu 
# apt-get install python3-dev graphviz libgraphviz-dev pkg-config

### Initialization model

In [7]:
# Install pygraphviz for visualizing
!pip install -qU pygraphviz==1.12

  error: subprocess-exited-with-error
  
  × Building wheel for pygraphviz (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [53 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build/lib.macosx-11.1-arm64-cpython-310/pygraphviz
      copying pygraphviz/scraper.py -> build/lib.macosx-11.1-arm64-cpython-310/pygraphviz
      copying pygraphviz/graphviz.py -> build/lib.macosx-11.1-arm64-cpython-310/pygraphviz
      copying pygraphviz/__init__.py -> build/lib.macosx-11.1-arm64-cpython-310/pygraphviz
      copying pygraphviz/agraph.py -> build/lib.macosx-11.1-arm64-cpython-310/pygraphviz
      copying pygraphviz/testing.py -> build/lib.macosx-11.1-arm64-cpython-310/pygraphviz
      creating build/lib.macosx-11.1-arm64-cpython-310/pygraphviz/tests
      copying pygraphviz/tests/test_unicode.py -> build/lib.macosx-11.1-arm64-cpython-310/pygraphviz/tests
      copying pygraphviz/tests/test_scraper.py -> build/lib.macosx-11.1-a

In [2]:
import vertexai
import os
import IPython
# load google access config file
credential_path="/Users/gongbiao/Code/vertex-ai/config/google_access_token_cp.json"
if os.path.exists(credential_path):
    print(f"the config load success")
else:
    print("config file does'not exists!")
    
# init vertex ai
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

config file does'not exists!


### Optional proxy setting

In [10]:
# [Optinal] set proxy
proxy = "http://127.0.0.1:8889"
os.environ["HTTP_PROXY"] = proxy
os.environ["HTTPS_PROXY"] = proxy
os.environ["http_proxy"] = proxy
os.environ["https_proxy"] = proxy

### Graph State
We will define a custom graph state to support our agent-oriented decision making. In this we will define:
- our user `input` (ie the most recent mesage from the user)
- `agent_out` which is used by the graph (and our final output) to consume/output agent outputs
- `intermediate_steps` which is a list maintained over our graph runtime to keep track of the results of previous

During each step in our graph we will be able to add to,modify, or extract these values from our state object.

In [ ]:
### Import LLM model

In [11]:
from typing import TypedDict, Annotated, List, Union
from langchain_google_vertexai import VertexAI
from langchain_core.agents import AgentAction, AgentFinish
import operator

class AgentState(TypedDict):
    input: str
    agent_out: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]


### Emulate Search 
To test a RAG-like agent we'll provide a tool that provide information as we would expect a search tool in a RAG agent to do.

In [13]:
ehi_information = """Title: EHI: End-to-end Learning of Hierarchical Index for
Efficient Dense Retrieval
Summary: Dense embedding-based retrieval is now the industry
standard for semantic search and ranking problems, like obtaining relevant web
documents for a given query. Such techniques use a two-stage process: (a)
contrastive learning to train a dual encoder to embed both the query and
documents and (b) approximate nearest neighbor search (ANNS) for finding similar
documents for a given query. These two stages are disjoint; the learned
embeddings might be ill-suited for the ANNS method and vice-versa, leading to
suboptimal performance. In this work, we propose End-to-end Hierarchical
Indexing -- EHI -- that jointly learns both the embeddings and the ANNS
structure to optimize retrieval performance. EHI uses a standard dual encoder
model for embedding queries and documents while learning an inverted file index
(IVF) style tree structure for efficient ANNS. To ensure stable and efficient
learning of discrete tree-based ANNS structure, EHI introduces the notion of
dense path embedding that captures the position of a query/document in the tree.
We demonstrate the effectiveness of EHI on several benchmarks, including
de-facto industry standard MS MARCO (Dev set and TREC DL19) datasets. For
example, with the same compute budget, EHI outperforms state-of-the-art (SOTA)
in by 0.6% (MRR@10) on MS MARCO dev set and by 4.2% (nDCG@10) on TREC DL19
benchmarks.
Author(s): Ramnath Kumar, Anshul Mittal, Nilesh Gupta, Aditya Kusupati,
Inderjit Dhillon, Prateek Jain
Source: https://arxiv.org/pdf/2310.08891.pdf"""

### Custom Tools
We wil define two tools for this agent, a `search` tool (which emulates our RAG component) and a `final_answer` tool -- which is 
providers output in a sepcific format. ie:
```json
{
    "answer": "<LLM generated answer here>",
    "source": "<LLM generated citation here>"
}
```

We define both using the `@tool` decorator from LangChain.

In [20]:
from langchain_core.tools import tool

@tool("search")
def search_tool(query: str) -> str:
    """Searches for information on the topic of artificial intelligence (AI).
    Cannot be used to research any other topics. Search query must be provided
    in natural language and be verbose."""
    # this is a "RAG" emulator
    return ehi_information

@tool("final_answer")
def final_answer_tool(answer:str, source: str) -> str:
    """Returns a natural language response to the user in `answer`, and a
    `source` which provides citations for where this information came from.
    """
    return ""
        

These tools will be triggered via OpenAI Tools (ie function calling). The LLM will be provided information on the schema (ie structure) of the function to be called, like that which we can see here:

In [21]:
search_tool

StructuredTool(name='search', description='Searches for information on the topic of artificial intelligence (AI).\n    Cannot be used to research any other topics. Search query must be provided\n    in natural language and be verbose.', args_schema=<class 'langchain_core.utils.pydantic.search'>, func=<function search_tool at 0x143014940>)

### Initalize Agent

In [ ]:
from typing import Annotated, Literal, TypedDict

from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
from langchain_google_vertexai import ChatVertexAI


# To use chat vertex ai model

inputs = {
    "input": "what are EHI embeddings?",
    "intermediate_steps": []
}
model = ChatVertexAI(model_name="gemini-1.5-pro-001", temperature=0)